# Modelo de n-gramas para determinar la polaridad del texto

## Estrategia

1. Elegir el modelo de n-gramas.

2. Crear un grupo de entrenamiento (85%) y otro de prueba (15%).

**Conjunto de entrenamiento:** 

3. Dentro del grupo de entrenamiento, crear dos subgrupos; el primero con las opiniones positivas y el segundo con las opiniones negativas.

4. Obtener los n-gramas de cada subgrupo. 

5. En una estructura tipo diccionario, a cada n-grama se le asigna un número entero que representa la cantidad de veces que dicho n-grama apareció en el subgrupo. No obstante, si el grupo es el negativo, entonces el número será negativo. En cambio, si el grupo es el positivo, entonces el número será positivo. 

6. Con los n-gramas que están presentes en ambos subgrupos, se tendrá que restar sus respectivos valores asociados y el resultado será el valor asociado final de dicho n-grama.

**Conjunto de prueba:** 
 
7. Para que el modelo asigne la polaridad a un texto del conjunto prueba, primero tendrá que obtener los n-gramas de dicho texto. 

8. A cada n-grama le asignará su valor asociado en el conjunto de entrenamiento.

9. Los n-gramas que no estén presentes en el conjunto de entrenamiento serán considerados <UNKNOWNS> (debatible).

10. Se realizará la suma de todos los valores asociados a los n-gramas del texto y el signo del resultado determinará la polaridad de dicho texto. 

11. Evaluar desempeño del modelo con F1-score. 


In [1]:
# Importar librerías
import pandas as pd
import nltk
from nltk import word_tokenize
from nltk.util import ngrams
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score, precision_score, recall_score
from collections import defaultdict
import string
from nltk.corpus import stopwords

In [2]:
# Descargar recursos necesarios de nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jcbar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jcbar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Función para cargar y preparar los datos

**Con esta función:**

1. Se cargan los datos desde un archivo CSV
2. Se concatenan los strings que se encuentran en la columna "Opinion" y en la columna "Title"
3. Se eliminan las filas que contienen nan's
4. Se separan los datos en un conjunto de entrenamiento (85%) y uno de prueba (15%)

In [3]:
# Función para cargar y preparar los datos
def load_and_prepare_data(file_path, test_size=0.15):
    """
    Carga el contenido de un archivo CSV y lo divide en conjuntos de prueba y entrenamiento.
    
    file_path: Ruta al archivo CSV
    
    return: Un DataFrame con los datos de entrenamiento y otro con los datos de prueba.
    """
    df = pd.read_csv(file_path)                                   # Cargar CSV
    df['Text'] = df['Title'] + " " + df['Opinion']                # Concatenar opinión y título
    df = df.dropna()                                              # Eliminar filas con nan's 
    
    return train_test_split(df, test_size=test_size, random_state=42)  # Divir datos en conjuntos de entrenamiento y prueba

### Función para limpiar el texto 

**Con esta función:**

1. De manera opcional, se pueden eliminar los signos de puntuación
2. De manera opcional,

    2.1. Todo el texto se pasa a mínusculas 

    2.2. Se realiza una tokenización del texto
    
    2.3  Se une el texto separando los tokens por espacios, ignorando las stopwords

In [4]:
# Función para eliminar signos de puntuación y stop words
def clean_text(text, remove_punctuation=True, remove_stopwords=True):
    """
    Limpia un texto (string) eliminando signos de puntuación y/o stopwords.
    
    text: Texto en formato string
    remove_punctuation: Booleano que determina si se elimina o no los signos de puntuación
    remove_stopword: Booleano que determina si se elimina o no las stopwords
    
    return: Texto (string) limpio.
    """
    if remove_punctuation:
        text = text.translate(str.maketrans('', '', string.punctuation))  # Eliminar los signos de puntuación: !"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~
    
    if remove_stopwords:
        stop_words = set(stopwords.words('spanish'))  # Extraer stopwords
        text = ' '.join([word for word in word_tokenize(text.lower()) if word not in stop_words])  # Tokenizar ignorando las stop-words
    
    return text

### Función para obtener los k-skip-n-gramas

**Con esta función:**

1. Se realiza una tokenización del texto
2. Se obtienen los k-skip-n-gramas del texto, con una k y n establecidas por el usuario

In [5]:
# Función para obtener n-gramas
def get_ngrams(text, n=2, skip=0):
    """
    Genera n-gramas (o skip-gramas si skip > 0) a partir de un texto.
    
    text: Texto a partir del cual se extraerán los n-gramas (o skip-gramas)
    n: Número de palabras en el n-grama
    skip: Número de palabras a saltar (solo para skip-gramas)
    
    return: Lista de n-gramas
    """
    tokens = word_tokenize(text.lower())  # Tokenizar el texto todo en mínusculas
    
    # Obterner n-gramas
    if skip == 0:
        return list(ngrams(tokens, n))
    
    # Obtener k-skip-n-gramas
    else:
        return list(nltk.skipgrams(tokens, n, skip))

### Función para obtener la frecuencia de aparición de n-gramas en un corpus

**Con esta función:**

1. Se obtienen los n-gramas de todos los textos de un corpus dado utilizando *get_ngrams*
2. Se cuenta la cantidad de veces que aparece cada n-grama en el corpus
3. Cada vez que un n-grama aparece en el corpus a su frecuencia de aparición se le suma el valor "weight" pre-determinado por el usuario

In [6]:
# Función para calcular las frecuencias de los n-gramas
def calculate_ngram_frequencies(texts, n=2, skip=0, weight=1):
    """
    Genera n-gramas (o skip-gramas si skip > 0) a partir de un conjunto de textos y calcula su frecuencia de aparición con base en weight.
    
    texts: Conjunto de textos de los que se obtendrán los n-gramas (o skip-gramas)
    n: Número de palabras en el n-grama
    skip: Número de palabras a saltar (solo para skip-gramas)
    weight: Cada aparición de un n-grama dado contribuirá con un valor de weight a su frecuencias
    
    return: defaultdict con la frecuencia de aparición de cada n-grama. 
    """

    ngram_values = defaultdict(int)  # Definir un defaultdict vacío
    
    # Iterar sobre todos los textos
    for text in texts:
        ngrams = get_ngrams(text, n, skip)  # Obtener n-gramas (o skip-gramas)
        
        # Iterar sobre todos los n-gramas
        for ngram in ngrams:
            ngram_values[ngram] += weight  # Sumar el valor weight a la frecuencia de cada n-grama cada vez que aparezca
    
    return ngram_values

### Función para realizar el cálculo de la frecuencia de aparición de cada n-grama utilizando un valor de weight específico

**Con esta función:**

1. Se utiliza *calculate_ngram_frequencies*  para ajustar el valor de weight, aplicando el hecho de que los n-gramas apareciendo en textos negativos reciben un valor de weight negativo y los n-gramas apareciendo en textos positivos reciben un valor positivo.
2. Además, dado el desbalance de las clases (solo el 5% son textos negativos), de manera opcional puede aplicarse un weight como sigue:

    weight= 1/len(textos_positivos) para los n-gramas apareciendo en textos positivos
    
    weight= -1/len(textos_negativos) para los n-gramas apareciendo en textos negativos 


3. Independientemente de los valores de weight utilizados, se suman las frecuencias de aparición de los n-gramas en el grupo positivo y negativo, dando como resultado una frecuencia neta para cada n-grama. 

In [7]:
# Función para reponderar las clases
def apply_class_weighting(positive_texts, negative_texts, n=2, skip=0, apply_weighting=True):
    """
    Realiza una re-pondercación de la frecuencia de aparición de cada n-grama dependiendo del grupo al que pertenece.
    
    positive_text: Textos calificados como positivos (Label = 1)
    negative_text: Textos calificados como negativos (Label = 0)
    n: Número de palabras en el n-grama
    skip: Número de palabras a saltar (solo para skip-gramas)
    apply_weighting: Booleano que determina si se desea ponderar las frecuencias de aparición de los n-gramas con base en el desbalance de clases
    
    return: defaultdict con la frecuencia de aparición re-ponderada de los n-gramas
    """
    
    num_positive = len(positive_texts)  # Cantidad de textos clasificados como positivos
    num_negative = len(negative_texts)  # Cantidad de textos clasificados como negativos
    
    weight_positive = 1 / num_positive if apply_weighting else 1  # Peso que tendrá la aparición de un n-grama en el grupo positivo
    weight_negative = 1 / num_negative if apply_weighting else 1  # Peso que tendrá la aparición de un n-grama en el grupo negativo
    
    # Calcular frecuencias de aparición en los grupos positivo y negativo con sus respectivos pesos
    positive_ngram_values = calculate_ngram_frequencies(positive_texts, n, skip, weight_positive)
    negative_ngram_values = calculate_ngram_frequencies(negative_texts, n, skip, -weight_negative)
    
    # Combinar n-gramas de ambos grupos sumando sus frecuencias respectivas
    combined_ngram_values = defaultdict(int, positive_ngram_values)
    for ngram, value in negative_ngram_values.items():
        combined_ngram_values[ngram] += value
    
    return combined_ngram_values

### Función para calcular la suma de las frecuencias netas de los n-gramas que componen a un texto

**Con esta función:** 

1. Se obtienen los n-gramas de un texto con *get_ngrams*
2. Cada n-grama se asocia con su valor de frecuencia neta calculado a partir de un weight y de su frecuencia de aparición en el grupo positivo y negativo 
3. Se suman las frecuencias netas de los n-gramas que componen al texto  

In [8]:
# Función para calcular la polaridad de un texto
def sum_net_frequencies(text, ngram_values, n=2, skip=0):
    """
    Obtiene los n-gramas (o skip-gramas sin skip > 0) de un texto y suma sus valores de frecuencia asociados. 
    
    text: Texto del que se obtendrán los n-gramas (o skip-gramas)
    ngram_values: defaultdict con los ngramas y sus valores de frecuencia neta respectivos 
    n: Número de palabras en el n-grama
    skip: Número de palabras a saltar (solo para skip-gramas)

    return: Suma total de los valores de frecuencias de todos los n-gramas que componen al texto
    """
    # Obtener n-gramas del texto
    ngrams = get_ngrams(text, n, skip)

    # Calcular la suma de los valores de frecuencia asociados a los n-gramas
    return sum(ngram_values[ngram] for ngram in ngrams)

### Función para predecir la polaridad de un texto

**Con esta función:** 

1. Se determina la polaridad del texto a partir del output de *sum_net_frequencies* 

    Si el signo del output es negativo, entonces es un texto negativo

    Si el signo del output es positivo, entonces es un texto positivo

In [9]:
# Función para predecir la polaridad en el conjunto de prueba
def predict_polarity(test_data, ngram_values, n=2, skip=0):
    """
    Determina la polaridad positiva/negativa de una serie de textos con base en el signo de la suma de la frecuencia neta de los n-gramas que componen a dichos textos. 
    
    test_data: Textos de los que se obtendrá su polaridad
    ngram_values: defaultdict con los ngramas y sus valores de frecuencia neta respectivos 
    n: Número de palabras en el n-grama
    skip: Número de palabras a saltar (solo para skip-gramas)

    return: 1 si la polaridad del texto es positiva y 0 si la polaridad es negativa
    """
    return [1 if sum_net_frequencies(text, ngram_values, n, skip) > 0 else 0 for text in test_data['Text']]

### Función para realizar todo un experimento con un modelo de n-grama dado

**Con esta función:** 

1. Se cargan y preparan los datos *load_and_prepare_data*, generando un corpus de entrenamiento y uno de prueba
2. De manera opcional, se remueven los signos de puntuación de los textos con *clean_data*
3. De manera opcional, se remueven las stopwords de los textos con *clean_data*
4. Con los textos de entrenamiento, se obtienen las frecuencias neta de los k-skip-n-gramas (con k y n determinados por el usuario) con base en su frecuencia de aparición en los textos positivos y negativos utilizando *apply_class_weighting*


     De manera opcional, se puede utilizar los valores de weight para abordar el problema del desbalance de clases


5. Con cada texto de prueba, se suman las frecuencias netas de sus k-skip-n-gramas y se predice su polaridad
6. Se mide el desempeño del modelo por medio de la F1-score 

In [10]:
# Función principal para entrenar y evaluar el modelo
def run_experiment(file_path, n=2, skip=0, remove_punctuation=True, remove_stopwords=True, apply_weighting=True):
    """
    Hace uso de las funciones definidas anteriormente para evaluar el desempeño de un modelo de n-gramas prediciendo la polaridad del conjunto de textos de prueba.
    
    file_path: Ruta al archivo CSV con los datos
    n: Número de palabras en el n-grama
    skip: Número de palabras a saltar (solo para skip-gramas)
    remove_punctuation: Booleano que determina si se elimina o no los signos de puntuación
    remove_stopword: Booleano que determina si se elimina o no las stopwords
    apply_weighting: Booleano que determina si se desea ponderar las frecuencias de aparición de los n-gramas con base en el desbalance de clases
    
    return: 1 si la polaridad del texto es positiva y 0 si la polaridad es negativa
    """
    
    # Cargar datos de entrenamiento y prueba
    train_data, test_data = load_and_prepare_data(file_path)
    
    # Limpiar datos (eliminar signos de puntuación y stopwords)
    train_data['Text'] = train_data['Text'].apply(lambda x: clean_text(x, remove_punctuation, remove_stopwords))
    test_data['Text'] = test_data['Text'].apply(lambda x: clean_text(x, remove_punctuation, remove_stopwords))
    
    # Separar textos positivos y negativos
    positive_texts = train_data[train_data['Label'] == 1]['Text']
    negative_texts = train_data[train_data['Label'] == 0]['Text']
    
    
    # Obtener las frecuencias netas de los n-gramas en el corpus
    ngram_values = apply_class_weighting(positive_texts, negative_texts, n, skip, apply_weighting)
    # Predecir la polaridad de los textos
    y_pred = predict_polarity(test_data, ngram_values, n, skip)
    
    # Evaluar el desempeño del modelo con f1-score
    return f1_score(test_data['Label'], y_pred)

### Función para realizar un Grid Search sobre los parámetros de la función *run_experiment*

**Con esta función:** 

1. Se ejecuta *run_experiment* con todas las combinaciones de valores posibles para sus parámetros
2. En cada caso se cálcula el valor de F1-score 
3. Se guarda el valor más alto de F1-score y sus los valores respectivos de los parámetros de *run_experiment*

In [11]:
# Grid Search Manual
def grid_search(file_path):
    """
    Realizar un grid search sobre los parámetros de la función run_experiment.
    Imprime el F1-score y los valores de los parámetros de run_experiment asociados. 
    
    file_path: Ruta al archivo CSV con los datos

    return: None
    """
    # Posibles valores de los n-gramas (nos limitamos a uni, bi y trigramas)
    n_values = [1, 2, 3]
    # Posibles valores de k en k-skip-n-gramas (nos limitamos a saltos de 1 y 2 tokens)
    skip_values = [0, 1, 2]
    # Posibles valores de los parámetros booleanos
    remove_punctuation_options = [True, False]
    remove_stopwords_options = [True, False]
    apply_weighting_options = [True, False]
    
    best_f1 = 0       # Variable para almacenar el valor más alto de F1-score
    best_params = {}  # Diccionario vacío para almacenar los mejores valores de los parámetros

    # Iteramos sorbre los posibles valores de los parámetros de run_experiment
    for n in n_values:
        for skip in skip_values:
            # Omitimos los casos en los que n==1 y skip > 0
            if n == 1 and skip > 0:
                continue
            for remove_punctuation in remove_punctuation_options:
                for remove_stopwords in remove_stopwords_options:
                    for apply_weighting in apply_weighting_options:
                        
                        # Ejecutamos run_experiment con un set de valores en sus parámetros
                        f1 = run_experiment(
                            file_path,
                            n=n,
                            skip=skip,
                            remove_punctuation=remove_punctuation,
                            remove_stopwords=remove_stopwords,
                            apply_weighting=apply_weighting
                        )
                        print(f"F1-score: {f1:.4f} | Params: n={n}, skip={skip}, remove_punctuation={remove_punctuation}, remove_stopwords={remove_stopwords}, apply_weighting={apply_weighting}")
                        
                        # En caso de que el valor actual de f1 sea mayor que el de best_f1, reemplazamos el valor de esta última
                        if f1 > best_f1:
                            best_f1 = f1
                            best_params = {
                                'n': n,
                                'skip': skip,
                                'remove_punctuation': remove_punctuation,
                                'remove_stopwords': remove_stopwords,
                                'apply_weighting': apply_weighting
                            }
    
    # Imprimimos el valor más alto de F1 y los valores respectivos de los parámetros de run_experiment
    print("\nBest F1-score:", best_f1)
    print("Best Parameters:", best_params)


In [12]:
# Ejecutar el Grid Search
grid_search('../Datos/train.csv')

F1-score: 0.7851 | Params: n=1, skip=0, remove_punctuation=True, remove_stopwords=True, apply_weighting=True
F1-score: 0.8987 | Params: n=1, skip=0, remove_punctuation=True, remove_stopwords=True, apply_weighting=False
F1-score: 0.0982 | Params: n=1, skip=0, remove_punctuation=True, remove_stopwords=False, apply_weighting=True
F1-score: 0.8987 | Params: n=1, skip=0, remove_punctuation=True, remove_stopwords=False, apply_weighting=False
F1-score: 0.0219 | Params: n=1, skip=0, remove_punctuation=False, remove_stopwords=True, apply_weighting=True
F1-score: 0.8987 | Params: n=1, skip=0, remove_punctuation=False, remove_stopwords=True, apply_weighting=False
F1-score: 0.0000 | Params: n=1, skip=0, remove_punctuation=False, remove_stopwords=False, apply_weighting=True
F1-score: 0.8987 | Params: n=1, skip=0, remove_punctuation=False, remove_stopwords=False, apply_weighting=False
F1-score: 0.8588 | Params: n=2, skip=0, remove_punctuation=True, remove_stopwords=True, apply_weighting=True
F1-scor

### A partir del Grid Search se identificó que el mejor modelo es un **2-skip-1-grama** actuando sobre textos **sin signos de puntuación**, **sin stopwords** y **con weights de 1 y -1** para los n-gramas apareciendo en textos positivos y negativos, respectivamente.  

Definimos una función para obtener las otras métricas (matriz de confusión, accuracy, recall y precision) de un modelo específico y con los datos siendo sometidos a un preprocesamiento determinado

In [73]:
# Función para evaluar el modelo bajo otras métricas
def evaluate_model(file_path, n, skip, remove_punctuation, remove_stopwords, apply_weighting):
    # Cargar y preparar los datos
    train_data, test_data = load_and_prepare_data(file_path)
    train_data['Text'] = train_data['Text'].apply(lambda x: clean_text(x, remove_punctuation=remove_punctuation, remove_stopwords=remove_stopwords))
    test_data['Text'] = test_data['Text'].apply(lambda x: clean_text(x, remove_punctuation=remove_punctuation, remove_stopwords=remove_stopwords))
    
    # Crear subgrupos de entrenamiento: positivos (Label=1) y negativos (Label=0)
    positive_texts = train_data[train_data['Label'] == 1]['Text']
    negative_texts = train_data[train_data['Label'] == 0]['Text']
    
    
    # Obtener las frecuencias netas de los n-gramas en el corpus
    ngram_values = apply_class_weighting(positive_texts, negative_texts, n, skip, apply_weighting=apply_weighting)

    # Predecir la polaridad de los textos en el conjunto de prueba
    y_pred = predict_polarity(test_data, ngram_values, n=n, skip=skip)
    
    # Evaluar el modelo
    y_test = test_data['Label']
    
    # Mostrar las métricas
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
    print(f"Precision: {precision_score(y_test, y_pred):.4f}")
    print(f"Recall: {recall_score(y_test, y_pred):.4f}")
    print(f"F1-score: {f1_score(y_test, y_pred):.4f}")
    print("\nConfusion Matrix:\n")
    print(confusion_matrix(y_test, y_pred))

- **2-skip-1-grama**

- **sin signos de puntuación**

- **sin stopwords**

- **weight = 1 en textos positivos**

- **weight = -1 en textos negativos**

In [74]:
# Ejecutar la evaluación del modelo
evaluate_model('../Datos/train.csv', n=2, skip=1, remove_punctuation=True, remove_stopwords=True, apply_weighting=False)

Accuracy: 0.8326
Precision: 0.8318
Recall: 0.9963
F1-score: 0.9066

Confusion Matrix:

[[ 13 109]
 [  2 539]]


- **2-skip-2-grama**

- **sin signos de puntuación**

- **sin stopwords**

- **weight = 1 en textos positivos**

- **weight = -1 en textos negativos**

In [75]:
# Ejecutar la evaluación del modelo
evaluate_model('../Datos/train.csv', n=2, skip=2, remove_punctuation=True, remove_stopwords=True, apply_weighting=False)

Accuracy: 0.8311
Precision: 0.8295
Recall: 0.9982
F1-score: 0.9060

Confusion Matrix:

[[ 11 111]
 [  1 540]]


- **0-skip-3-grama**

- **con signos de puntuación**

- **con stopwords**

- **weight = 1 en textos positivos**

- **weight = -1 en textos negativos**

In [76]:
# Ejecutar la evaluación del modelo
evaluate_model('../Datos/train.csv', n=3, skip=0, remove_punctuation=False, remove_stopwords=False, apply_weighting=False)

Accuracy: 0.8190
Precision: 0.8194
Recall: 0.9982
F1-score: 0.9000

Confusion Matrix:

[[  3 119]
 [  1 540]]


- **3-skip-2-grama**

- **sin signos de puntuación**

- **con stopwords**

- **weight = 1 en textos positivos**

- **weight = -1 en textos negativos**

In [77]:
# Ejecutar la evaluación del modelo
evaluate_model('../Datos/train.csv', n=3, skip=2, remove_punctuation=True, remove_stopwords=False, apply_weighting=False)

Accuracy: 0.8190
Precision: 0.8194
Recall: 0.9982
F1-score: 0.9000

Confusion Matrix:

[[  3 119]
 [  1 540]]
